In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time 
from selenium.webdriver.support.ui import Select
import pandas as pd
import openpyxl
import re
import os
import glob
driver = webdriver.Chrome()
driver.get('https://scholar.google.com/citations?view_op=search_authors')
search_bttn = driver.find_element(By.XPATH, '//*[@id="gs_hdr_tsi"]')
search_bttn.send_keys("Radwan,A.")
click_search_btnn =   driver.find_element(By.XPATH, '//*[@id="gs_hdr_tsb"]/span/span[1]').click()
click_on_author = driver.find_element(By.XPATH, '//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3/a').click()
publication_links = driver.find_elements(By.CSS_SELECTOR, '#gsc_a_b .gsc_a_t a')
print(publication_links)
click_on_first_publication =  driver.find_element(By.XPATH, '//*[@id="gsc_a_b"]/tr[1]/td[1]/a').click()
click_on_pub_link =  driver.find_element(By.XPATH,'//*[@id="gsc_oci_title_gg"]/div').click()
click_on_email_icon = driver.find_element(By.XPATH,'//*[@id="author-group"]/button[1]/span/span[1]').click()
copy_email =  driver.find_element(By.XPATH,'//*[@id="side-panel-author"]/div[3]/a/span').text
print(copy_email)
click_on_email_icon_2 = driver.find_element(By.XPATH, '//*[@id="author-group"]/button[2]').click()
copy_email_2 = driver.find_element(By.XPATH,'//*[@id="side-panel-author"]/div[4]/a/span').text
print(copy_email_2)

There was an error managing chromedriver (request or response body error: operation timed out); using driver found in the cache


[<selenium.webdriver.remote.webelement.WebElement (session="85741754bb4f12321ed795ef1911cf38", element="f.EE179F6E58ECAB60C83345E81C58C7BF.d.37A8DF96EE9F2B0AF45DBF0030B4D946.e.47")>, <selenium.webdriver.remote.webelement.WebElement (session="85741754bb4f12321ed795ef1911cf38", element="f.EE179F6E58ECAB60C83345E81C58C7BF.d.37A8DF96EE9F2B0AF45DBF0030B4D946.e.48")>, <selenium.webdriver.remote.webelement.WebElement (session="85741754bb4f12321ed795ef1911cf38", element="f.EE179F6E58ECAB60C83345E81C58C7BF.d.37A8DF96EE9F2B0AF45DBF0030B4D946.e.49")>, <selenium.webdriver.remote.webelement.WebElement (session="85741754bb4f12321ed795ef1911cf38", element="f.EE179F6E58ECAB60C83345E81C58C7BF.d.37A8DF96EE9F2B0AF45DBF0030B4D946.e.50")>, <selenium.webdriver.remote.webelement.WebElement (session="85741754bb4f12321ed795ef1911cf38", element="f.EE179F6E58ECAB60C83345E81C58C7BF.d.37A8DF96EE9F2B0AF45DBF0030B4D946.e.51")>, <selenium.webdriver.remote.webelement.WebElement (session="85741754bb4f12321ed795ef1911cf

In [7]:
import requests
import re

def extract_strings_with_at_symbol(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  
        if response.status_code == 200:
            html_content = response.text
            at_symbol_pattern = r'\b\S*@\S*\b'
            strings_with_at_symbol = re.findall(at_symbol_pattern, html_content)
            if strings_with_at_symbol:
             
                return strings_with_at_symbol
            else:
                return []
        else:
            print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during request: {e}")
        return []
    except Exception as e:
        print(f"Error occurred while extracting strings: {e}")
        return []

if __name__ == "__main__":

    website_url = "https://uni-tuebingen.de/fakultaeten/mathematisch-naturwissenschaftliche-fakultaet/fachbereiche/informatik/lehrstuehle/algorithms-in-bioinformatics/"
    extracted_strings = extract_strings_with_at_symbol(website_url)
    if extracted_strings:
        print("Strings containing '@' found on the website:")
        for string in extracted_strings:
            print(string)
    else:
        print("No strings containing '@' found on the website.")


Strings containing '@' found on the website:
value="146,147,166,8290,8291,8394,8395,8425,8426,9608,31267,8293,8366@c40ba2f920a5ca0bb18bcdfcdc389e757e48a010
ut-nav__link--level-3">Hochschulweiterbildung@BW</a></li><li
class="mail">daniel.huson@.uni-tuebingen.de</a></p
prevention</span>@uni-tuebingen.de</a></p
href="https://xn--baw-joa.social/@unituebingen


In [17]:
import requests
import pandas as pd
import numpy as np

def get_author_info(author_id, api_key):
    url = f'https://api.elsevier.com/content/search/author?co-author={author_id}'
    #url = f'https://api.elsevier.com/content/search/author?co-author=(57062645000)&apiKey=3334155c411aa69fa7f1328e0c50ad7c'
    headers = {'Accept': 'application/json', 'X-ELS-APIKey': api_key}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        print(data)
        print("Good response")
        return data
    else:
        print(f"Failed to retrieve author information. Status code: {response.status_code}")
        return None
    
def extract_coauthors_info(author_data):
    if 'search-results' in author_data:
        author_info = author_data['search-results']['opensearch:Query']
        coauthors_info = []
        if '@searchTerms' in author_info:
            documents = author_info['@searchTerms']
            print(documents)
            for document in documents:
                coauthors_info.append({'coauthor_id': document})
    
        return coauthors_info
    else:
        return None, []

def save_coauthors_to_csv(coauthors_info, output_file):
    data = []
    for coauthor in coauthors_info:
        data.append([coauthor])

    df = pd.DataFrame(data, columns=['Co-author IDs'])
    df.to_csv(output_file, index=False)

if __name__ == "__main__":
    api_key = '3334155c411aa69fa7f1328e0c50ad7c'
    author_id = '57062645000'
    output_file = 'coauthors_info.csv'
    author_data = get_author_info(author_id, api_key)
    if author_data:
        coauthors_info = extract_coauthors_info(author_data)

        if coauthors_info:
            save_coauthors_to_csv(coauthors_info, output_file)
            print(f"Co-authors' information saved to '{output_file}'")
        else:
            print("No co-authors found for the author.")
    else:
        print("Failed to retrieve author information.")


{'search-results': {'opensearch:totalResults': '154', 'opensearch:startIndex': '0', 'opensearch:itemsPerPage': '25', 'opensearch:Query': {'@role': 'request', '@searchTerms': '(au-id(57062645000) OR au-id(6604069881) OR au-id(57224767865) OR au-id(15924778900) OR au-id(6603536955) OR au-id(53984497000) OR au-id(55862150300) OR au-id(57204779310) OR au-id(57207470426) OR au-id(57212062515) OR au-id(16246088000) OR au-id(35234647000) OR au-id(53979981400) OR au-id(54409464800) OR au-id(55432866400) OR au-id(55512788100) OR au-id(55627327200) OR au-id(55673597300) OR au-id(55951661000) OR au-id(56060998600) OR au-id(56331067900) OR au-id(56385148800) OR au-id(57190402049) OR au-id(57215599158) OR au-id(57215609785) OR au-id(57215609995) OR au-id(57215611982) OR au-id(57218547898) OR au-id(57218629162) OR au-id(57732413200) OR au-id(58207442100) OR au-id(13104617000) OR au-id(16047047000) OR au-id(16305821100) OR au-id(16548876700) OR au-id(23098155200) OR au-id(24172264700) OR au-id(256411

In [39]:
import requests
import pandas as pd

def get_author_info(author_id, api_key):
    url = f'https://api.elsevier.com/content/search/author?co-author={author_id}'
    headers = {'Accept': 'application/json', 'X-ELS-APIKey': api_key}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        print(data)
        print("Good response")
        return data
    else:
        print(f"Failed to retrieve author information. Status code: {response.status_code}")
        return None

def extract_coauthors_info(author_data):
    author_ids = []
        
    if 'search-results' in author_data and 'opensearch:Query'in author_data['search-results']:
        entries = author_data['search-results']['opensearch:Query']
        
        for entry in entries:
            if '@searchTerms' in entry:
                author_ids.append(entries['@searchTerms'])
    
    for i in author_ids:
        if 'OR' in i:
            i.replace("OR", ',')
    return author_ids


def save_coauthors_to_csv(coauthors_info, output_file):
    if not coauthors_info:
        print("No co-authors found for the author.")
        return
    
    df = pd.DataFrame(coauthors_info)
    df.to_csv(output_file, index=False)
    print(f"Co-authors' information saved to '{output_file}'")

if __name__ == "__main__":
    api_key = '3334155c411aa69fa7f1328e0c50ad7c'
    author_id = '57062645000'
    output_file = 'coauthors_info.csv'
    
    author_data = get_author_info(author_id, api_key)
    
    if author_data:
        coauthors_info = extract_coauthors_info(author_data)
        
        if coauthors_info is not None:
            save_coauthors_to_csv(coauthors_info, output_file)
        else:
            print("Failed to extract co-authors' information.")
    else:
        print("Failed to retrieve author information.")


{'search-results': {'opensearch:totalResults': '154', 'opensearch:startIndex': '0', 'opensearch:itemsPerPage': '25', 'opensearch:Query': {'@role': 'request', '@searchTerms': '(au-id(57062645000) OR au-id(6604069881) OR au-id(57224767865) OR au-id(15924778900) OR au-id(6603536955) OR au-id(53984497000) OR au-id(55862150300) OR au-id(57204779310) OR au-id(57207470426) OR au-id(57212062515) OR au-id(16246088000) OR au-id(35234647000) OR au-id(53979981400) OR au-id(54409464800) OR au-id(55432866400) OR au-id(55512788100) OR au-id(55627327200) OR au-id(55673597300) OR au-id(55951661000) OR au-id(56060998600) OR au-id(56331067900) OR au-id(56385148800) OR au-id(57190402049) OR au-id(57215599158) OR au-id(57215609785) OR au-id(57215609995) OR au-id(57215611982) OR au-id(57218547898) OR au-id(57218629162) OR au-id(57732413200) OR au-id(58207442100) OR au-id(13104617000) OR au-id(16047047000) OR au-id(16305821100) OR au-id(16548876700) OR au-id(23098155200) OR au-id(24172264700) OR au-id(256411

In [41]:

import requests
import pandas as pd

def get_author_info(author_id, api_key):
    url = f'https://api.elsevier.com/content/search/author?co-author={author_id}'
    headers = {'Accept': 'application/json', 'X-ELS-APIKey': api_key}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        print("Good response")
        return data
    else:
        print(f"Failed to retrieve author information. Status code: {response.status_code}")
        return None

def extract_coauthors_info(author_data):
    author_ids = []
        
    if 'search-results' in author_data and 'opensearch:Query' in author_data['search-results']:
        query_info = author_data['search-results']['opensearch:Query']
        search_terms = query_info.get('@searchTerms', '')
        
        # Split the search terms by "OR" to extract author IDs
        author_ids = [term.strip() for term in search_terms.split('OR')]
    
    # Replace "OR" with "," in each author ID
    author_ids = [author_id.replace("OR", ",") for author_id in author_ids]
    
    return author_ids

def save_coauthors_to_csv(coauthors_info, output_file):
    if not coauthors_info:
        print("No co-authors found for the author.")
        return
    
    df = pd.DataFrame({'Co-author IDs': coauthors_info})
    df.to_csv(output_file, index=False)
    print(f"Co-authors' information saved to '{output_file}'")

if __name__ == "__main__":
    api_key = '3334155c411aa69fa7f1328e0c50ad7c'
    author_id = '57062645000'
    output_file = 'coauthors_info.csv'
    
    author_data = get_author_info(author_id, api_key)
    
    if author_data:
        coauthors_info = extract_coauthors_info(author_data)
        
        if coauthors_info is not None:
            save_coauthors_to_csv(coauthors_info, output_file)
        else:
            print("Failed to extract co-authors' information.")
    else:
        print("Failed to retrieve author information.")


Good response
Co-authors' information saved to 'coauthors_info.csv'


In [62]:
import requests
import pandas as pd

def get_author_info(author_id, api_key):
    url = f'https://api.elsevier.com/content/search/author?co-author={author_id}'
    headers = {'Accept': 'application/json', 'X-ELS-APIKey': api_key}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        print("Good response")
        return data
    else:
        print(f"Failed to retrieve author information. Status code: {response.status_code}")
        return None

def extract_coauthors_info(author_data):
    author_ids = []
        
    if 'search-results' in author_data and 'opensearch:Query' in author_data['search-results']:
        query_info = author_data['search-results']['opensearch:Query']
        search_terms = query_info.get('@searchTerms', '')
        
        # Split the search terms by "OR" to extract author IDs
        author_ids = [term.strip() for term in search_terms.split('OR')]
    
    # Replace "OR" with "," in each author ID and remove "au-id("
    cleaned_author_ids = []
    for author_id in author_ids:
        cleaned_author_id = author_id.replace("OR", ",").replace("au-id(", "").replace(")", "").replace("(","")
        cleaned_author_ids.append(cleaned_author_id)
    # cleaned_author_ids = [str(author_id) for author_id in coauthors_info]
    return cleaned_author_ids

def save_coauthors_to_csv(coauthors_info, output_file):
    # if not coauthors_info:
    #     print("No co-authors found for the author.")
    #     return
    
    # df = pd.DataFrame({'Co-author IDs': coauthors_info})
    # df['Co-author IDs'] = df['Co-author IDs'].astype(str)
    # df.to_csv(output_file, index=False)
    # print(f"Co-authors' information saved to '{output_file}'")
 

    if not coauthors_info:
        print("No co-authors found for the author.")
        return
    
    # Convert all author IDs to strings
    cleaned_author_ids = [str(author_id) for author_id in coauthors_info]
    
    # Create a DataFrame with the cleaned author IDs as strings
    df = pd.DataFrame({'Co-author IDs': cleaned_author_ids})
    
    # Save DataFrame to CSV file with string values
    df.to_csv(output_file, index=False, quoting=1)  # Use quoting=1 for non-numeric values
    
    print(f"Co-authors' information saved to '{output_file}'")

if __name__ == "__main__":
    api_key = '3334155c411aa69fa7f1328e0c50ad7c'
    author_id = '57062645000'
    output_file = 'coauthors.csv'
    
    author_data = get_author_info(author_id, api_key)
    
    if author_data:
        coauthors_info = extract_coauthors_info(author_data)
        
        if coauthors_info is not None:
            save_coauthors_to_csv(coauthors_info, output_file)
        else:
            print("Failed to extract co-authors' information.")
    else:
        print("Failed to retrieve author information.")


Good response
Co-authors' information saved to 'coauthors_info.csv'
